# Data Cleaning
In this notebook, we inspect the provided data files to identify data cleaning needs and execute all data cleaning tasks. Cleaned data is then exported to the data_cleaned folder for consumption by the analysis notebooks.

## Initial setup

### Importing libraries

In [1]:
import pandas as pd

### Data inspection function
Below we define the function `inspect_dataframe`, which provides a summary of the data contained in the input data frame.

In [2]:
def inspect_dataframe(df):
    print(f"There are {len(df)} rows in this data frame.\n")
    print('Column Names and Types:\n')
    print(df.dtypes,'\n')
    print('The table below shows the number of missing values in each column:\n')
    print(df.isna().sum(),'\n')
    for columnName in df.columns:
        if df[columnName].dtype in (object, str):
            print(f"There are {len(df[columnName].unique())} unique values in {columnName}. The first five are listed below:\n")
            for value in df[columnName].unique()[:5]:
                print(value,)
            print('\n')
        elif df[columnName].dtype in (int, float):
            column_min = df[columnName].min()
            column_max = df[columnName].max()
            print(f"The column {columnName} ranges between a minimum value of {column_min} and a maximum value of {column_max}.\n")

## The `imdb.title.basics.csv` data source

### Loading `imdb.title.basics.csv`

In [3]:
title_basics_df = pd.read_csv('data/imdb.title.basics.csv')

### Inspecting `imdb.title.basics.csv`
From the summary below we note the following initial impressions:
 * The variable `tconst` appears to be a primary key.
 * The variable `primary_title` is the expected title of the movie formated as a string.
 * The variable `original_title` seems like it might encode the name of the original release of a movie that has been re-released. This may be most useful when the original title is in the original language of the movie.
 * The variable `start_year` ranges from 2010 to 2115, indicating that this column most likely needs to inspected more closely to identify malformed data.
 * The variable `runtime_minutes` ranges from 1 to 51420, indicating that this column most likely needs to inspected more closely to identify malformed data.
 * The variable `genres` contains a comma separated list of applicable genres. We should identify a list of unique genres and re-encode these lists as distinct indicator variables. 

In [4]:
inspect_dataframe(title_basics_df)

There are 146144 rows in this data frame.

Column Names and Types:

tconst              object
primary_title       object
original_title      object
start_year           int64
runtime_minutes    float64
genres              object
dtype: object 

The table below shows the number of missing values in each column:

tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64 

There are 146144 unique values in tconst. The first five are listed below:

tt0063540
tt0066787
tt0069049
tt0069204
tt0100275


There are 136071 unique values in primary_title. The first five are listed below:

Sunghursh
One Day Before the Rainy Season
The Other Side of the Wind
Sabse Bada Sukh
The Wandering Soap Opera


There are 137774 unique values in original_title. The first five are listed below:

Sunghursh
Ashad Ka Ek Din
The Other Side of the Wind
Sabse Bada Sukh
La Telenovela Errante


The column start_year

### Cleaning `imdb.title.basic.csv`

#### Dropping movies with `start_year` in the future
Below we drop rows with movies that have a `start_year` after 2019. This ensures that our analysis covers full years and avoids incorporating data from unreleased movies. 

In [5]:
cleaned_title_basics_df = title_basics_df.query('start_year <= 2019')

#### Inspecting movies with extreme `runtime_minutes`
Manually verifying the runtime of a few of the longest movies in the list shows that these values are accurate. There is no need to clean this column. 


In [6]:
title_basics_df.query('runtime_minutes > 240').sort_values('runtime_minutes')

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
3918,tt10374170,Friends with Awkwardness,Friends with Awkwardness,2019,241.0,Comedy
118879,tt7131170,Rideshare,Rideshare,2016,242.0,"Horror,Thriller"
77066,tt4417796,Les choses et les mots de Mudimbe,Les choses et les mots de Mudimbe,2015,243.0,Documentary
36099,tt2321493,Romanian Art Scene 2009-2011,Romanian Art Scene 2009-2011,2012,243.0,Documentary
125405,tt7639166,Just Tell Her,Just Tell Her,2017,243.0,Action
...,...,...,...,...,...,...
88717,tt5136218,London EC1,London EC1,2015,5460.0,"Comedy,Drama,Mystery"
87264,tt5068890,Hunger!,Hunger!,2015,6000.0,"Documentary,Drama"
123467,tt7492094,Nari,Nari,2017,6017.0,Documentary
44840,tt2659636,Modern Times Forever,Modern Times Forever,2011,14400.0,Documentary


#### Re-encoding the `genres` variable
We noticed that `genres` is a, potentially empty, list of applicable genres. Below we re-encode this data using indicator variables.

In [7]:
def get_unique_genres(df):
    unique_genres = set()
    for title_genres in df['genres']:
        try:
            for genre in title_genres.split(','):
                unique_genres.add(genre)
        except:
            continue
    unique_genres_list = sorted(list(unique_genres))
    return unique_genres_list

In [8]:
def genre_indicator(title_genres, genre):
    try:
        title_genres_list = title_genres.split(',')
        if genre in title_genres_list:
            value = True
        else:
            value = False
    except:
        value = False
    return value

In [9]:
def test_genre_indicator():
    test_title_genres = "Drama,Comedy"
    test_genre1 = "Drama"
    test_genre2 = "Musical"
    return genre_indicator(test_title_genres, test_genre1) == True and genre_indicator(test_title_genres, test_genre2) == False

test_genre_indicator()

True

In [10]:
def add_genre_indicators(df):
    unique_genres = get_unique_genres(df)
    for genre in unique_genres:
        genre_name = genre.lower().replace('-','_')
        column_name = "genre_"+genre_name
        df[column_name] = df['genres'].apply(lambda x: genre_indicator(x, genre))
    return df

In [11]:
add_genre_indicators(cleaned_title_basics_df)

/home/schart/anaconda3/envs/flatiron/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre_action,genre_adult,genre_adventure,genre_animation,...,genre_news,genre_reality_tv,genre_romance,genre_sci_fi,genre_short,genre_sport,genre_talk_show,genre_thriller,genre_war,genre_western
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,NaN,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Inspecting cleaned data

In [17]:
inspect_dataframe(cleaned_title_basics_df)

There are 145081 rows in this data frame.

Column Names and Types:

tconst                object
primary_title         object
original_title        object
start_year             int64
runtime_minutes      float64
genres                object
genre_action            bool
genre_adult             bool
genre_adventure         bool
genre_animation         bool
genre_biography         bool
genre_comedy            bool
genre_crime             bool
genre_documentary       bool
genre_drama             bool
genre_family            bool
genre_fantasy           bool
genre_game_show         bool
genre_history           bool
genre_horror            bool
genre_music             bool
genre_musical           bool
genre_mystery           bool
genre_news              bool
genre_reality_tv        bool
genre_romance           bool
genre_sci_fi            bool
genre_short             bool
genre_sport             bool
genre_talk_show         bool
genre_thriller          bool
genre_war               bool
genr

## The `imdb.title.ratings.csv` data source

### Loading the `imdb.title.ratings.csv` data source

In [13]:
title_ratings_df = pd.read_csv('data/imdb.title.ratings.csv')

### Inspecting `imdb.title.ratings.csv`
This file seems to contain ratings information for the films listed in `imdb.title.basic.csv` data source. All of the variables seem to be properly formatted and there are no missing values. This source requires no further action. 

In [14]:
inspect_dataframe(title_ratings_df)

There are 73856 rows in this data frame.

Column Names and Types:

tconst            object
averagerating    float64
numvotes           int64
dtype: object 

The table below shows the number of missing values in each column:

tconst           0
averagerating    0
numvotes         0
dtype: int64 

There are 73856 unique values in tconst. The first five are listed below:

tt10356526
tt10384606
tt1042974
tt1043726
tt1060240


The column averagerating ranges between a minimum value of 1.0 and a maximum value of 10.0.

The column numvotes ranges between a minimum value of 5 and a maximum value of 1841066.



## The `bom.movie_gross.csv` data source

### Loading the `bom.movie_gross.csv` data source

In [15]:
title_ratings_df = pd.read_csv('data/bom.movie_gross.csv')

### Inspecting `bom.movie_gross.csv`
From the summary below we note the following initial impressions:
 * The `title` variable seems to be properly formatted and is nearly unique. Further analysis should identify the one duplicate value and determine if the year separates the duplicate titles.
 * The `studio` variable is has very few missing values. It seems to be formatted as an abbreviation of the studio name. It would be nice to identify the full names of studios and map this variable to one that provides the full name.
 * The `domestic_gross` variable appears to be properly formatted with a small number of missing values. The maximum value is large but plausible. This variable should be inspected for outliers. It may be necessary to correct this variable for inflation.  
 * The `foreign_gross` variable seems to be improperly formatted as a string. This should be converted to a float after confirming that there is a uniform currency. 
 * The `year` variable is properly formatted and has no missing values. 
 
Since this data source does not share a key with the imdb sources, it will be necessary to match titles, most likely on the combination of title and year.  

In [16]:
inspect_dataframe(title_ratings_df)

There are 3387 rows in this data frame.

Column Names and Types:

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object 

The table below shows the number of missing values in each column:

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64 

There are 3386 unique values in title. The first five are listed below:

Toy Story 3
Alice in Wonderland (2010)
Harry Potter and the Deathly Hallows Part 1
Inception
Shrek Forever After


There are 258 unique values in studio. The first five are listed below:

BV
WB
P/DW
Sum.
Par.


The column domestic_gross ranges between a minimum value of 100.0 and a maximum value of 936700000.0.

There are 1205 unique values in foreign_gross. The first five are listed below:

652000000
691300000
664300000
535700000
513900000


The column year ranges between a minimum value of 2010 and a maximum v